In [1]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
#  selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver
# selenium으로 무엇인가 입력하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
from selenium.webdriver.chrome.options import Options
import pandas as pd
from pyshadow.main import Shadow

options = Options()

options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/120.0.0.0 Safari/537.36"
)


In [2]:
def scroll_and_crawl(driver, target_count=100):
    name_list, skin_list, star_list = [], [], []
    date_list, good_list, text_list = [], [], []
    collected_reviews = set()
    
    # 바닥 확인 실패 횟수 카운트 (연속 3번 바닥이면 진짜 종료)
    no_new_count = 0 
    
    while len(name_list) < target_count:
        shadow = Shadow(driver)
        users = shadow.find_elements('div.inner')
        
        initial_count = len(name_list) # 루프 시작 전 수집 개수
        
        for user in users:
            try:
                user_hosts = user.find_elements(By.CSS_SELECTOR, 'oy-review-review-user')
                text_hosts = user.find_elements(By.CSS_SELECTOR, 'oy-review-review-content')
                
                if not user_hosts or not text_hosts:
                    continue
                
                text_shadow = text_hosts[0].shadow_root
                t_el = text_shadow.find_elements(By.CSS_SELECTOR, 'div.content')
                review_text = t_el[0].text.strip() if t_el else ""
                
                # 중복 체크 및 빈 내용 제외
                if not review_text or review_text in collected_reviews:
                    continue 
                
                # ... [데이터 추출 부분은 기존 코드와 동일] ...
                user_host = user_hosts[0]
                user_shadow = user_host.shadow_root
                n_el = user_shadow.find_elements(By.CSS_SELECTOR, 'div.info div.name')
                s_el = user_shadow.find_elements(By.CSS_SELECTOR, 'div.info div.skin-types')
                star_rating = user.find_elements(By.CSS_SELECTOR, "div.meta div.rating oy-review-star-icon")
                date = user.find_elements(By.CSS_SELECTOR, 'div.common-info span.date')
                goods_option = user.find_elements(By.CSS_SELECTOR, 'div.goods-option')

                name_list.append(n_el[0].text if n_el else "이름없음")
                skin_list.append(s_el[0].text.replace('\n', ',') if s_el else None)
                text_list.append(review_text.replace('\n', ' '))
                star_list.append(len(star_rating))
                date_list.append(date[0].text if date else None)
                good_list.append(goods_option[0].text if goods_option else None)

                collected_reviews.add(review_text)
                
                if len(name_list) >= target_count: break
            except:
                continue

        # --- 개선된 스크롤 및 바닥 체크 로직 ---
        driver.execute_script("window.scrollBy(0, 2000);") # 스크롤 폭을 조금 더 크게
        time.sleep(4) # 네트워크 속도에 따라 3~4초 권장
        
        # 수집된 개수가 늘지 않았다면?
        if len(name_list) == initial_count:
            no_new_count += 1
            print(f"새로운 리뷰 로딩 대기 중... ({no_new_count}/3)")
        else:
            no_new_count = 0 # 새 데이터 수집되면 카운트 초기화
            
        if no_new_count >= 3: # 3번 연속으로 새 리뷰가 없으면 종료
            print("더 이상 불러올 리뷰가 없어 수집을 종료합니다.")
            break
            
        print(f"현재 수집된 리뷰: {len(name_list)}개...")

    return name_list, skin_list, star_list, date_list, good_list, text_list

In [3]:
info = pd.read_pickle('화장품최종본.plk')
name2 = info.loc[:,'이름']
links = info.loc[:,'링크'] + ['&tab=review']
links
ratings = info.loc[:,'리뷰수'].str.replace(",", "").str.extract('([0-9]+)')[0].astype(int).tolist()


In [ ]:
%%time
all_data =[]

link_list = links.to_list()
name_list = name2.to_list()


driver = webdriver.Chrome(options=options) 
driver.maximize_window()

for name, link, rate in zip(name_list, link_list, ratings):
    try:
        print(f"\n >>> {name} 상품 수집 시작")
        print(f"링크: {link}")
        
        # 해당 상품 페이지 접속
        driver.get(link)
        time.sleep(3) # 페이지 로딩 대기

        # 3. 작성하신 함수 호출 (목표 개수 100개)
        names, skins, stars, dates, goods, texts = scroll_and_crawl(driver, target_count=rate)

        # 4. 수집된 데이터를 리스트에 임시 저장
        for j in range(len(names)):
            all_data.append({
                '상품이름': name,
                '상품링크': link,
                '작성자': names[j],
                '피부타입': skins[j],
                '별점': stars[j],
                '날짜': dates[j],
                '옵션': goods[j],
                '리뷰내용': texts[j]
            })
            
        print(f"성공: {len(names)}개의 리뷰 수집 완료")
            
    except Exception as e:
        print(f"오류 발생: {link}에서 데이터 수집 중 문제가 생겼습니다. {e}")
        continue

df =pd.DataFrame(all_data)



In [ ]:
df2 =  pd.read_pickle('화장품리뷰.plk')

In [5]:
df2.loc[:,'상품이름'].value_counts()


상품이름
오브제 내추럴 커버 파운데이션 기획세트(+클렌징폼 50ml)                   6452
[덱스 PICK] 오브제 내추럴 커버 파운데이션 13g (SPF50+, PA++++)     3677
비레디 블루 쿠션 4세대 15g(SPF 34 PA++)                      3250
비레디 블루 쿠션 4세대 리필 15g                                1676
[덱스 PICK]오브제 퍼펙트 옴므 쿠션 15g (SPF50+PA+++)             955
비레디 블루 파운데이션 35ml                                    855
다슈 맨즈 멀티 커버 스틱 파운데이션 ( 단품/기획 )                       643
다슈 맨즈 멀티 커버 스틱 파운데이션 12g 뉴트럴베이지 기획(+미니 톤 컨트롤 로션)     643
[맨즈파데1등] 그라펜 팁 스케치 멀티 커버 파운데이션 20ml                  494
[착붙쿠션] 그라펜 핸섬 커버쿠션 2종 택1                             430
[피부착붙컬러] 그라펜 매치업 파운데이션 40ml 단품/기획                    227
두잉왓 소프트 블러링 필터 쿠션 15g 단품/기획                          217
다슈 맨즈 아쿠아 매트 비비 쿠션 15g                                60
다슈 맨즈 아쿠아 매트 비비 쿠션 (리필용)                              22
다슈 맨즈 아쿠아 매트 비비쿠션 (본품+리필)                              8
Name: count, dtype: int64

In [6]:
ratings

[6892, 883, 974, 3322, 671, 576, 452, 3832, 1717, 61, 230, 23, 8, 671, 219]

그라펜 1에 "좋아요!"
그라펜 2에 "좋아요!"
가 중복이여서 카운트가 많이됌

각 리뷰내에서 중복 내용제거

In [12]:
import re

df2['리뷰내용_정제'] = (
    df2['리뷰내용']
    .str.replace(r'[^\w\s]', '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)
for i in df2['상품이름'].unique():
    cnt = (
        df2[df2['상품이름'] == i]
        .duplicated(subset=['작성자', '리뷰내용_정제'])
        .sum()
    )
    print(i, cnt)



오브제 내추럴 커버 파운데이션 기획세트(+클렌징폼 50ml) 8
비레디 블루 파운데이션 35ml 0
[덱스 PICK]오브제 퍼펙트 옴므 쿠션 15g (SPF50+PA+++) 2
비레디 블루 쿠션 4세대 15g(SPF 34 PA++) 2
다슈 맨즈 멀티 커버 스틱 파운데이션 ( 단품/기획 ) 0
[맨즈파데1등] 그라펜 팁 스케치 멀티 커버 파운데이션 20ml 0
[착붙쿠션] 그라펜 핸섬 커버쿠션 2종 택1 1
[덱스 PICK] 오브제 내추럴 커버 파운데이션 13g (SPF50+, PA++++) 4
비레디 블루 쿠션 4세대 리필 15g 1
다슈 맨즈 아쿠아 매트 비비 쿠션 15g 0
[피부착붙컬러] 그라펜 매치업 파운데이션 40ml 단품/기획 0
다슈 맨즈 아쿠아 매트 비비 쿠션 (리필용) 0
다슈 맨즈 아쿠아 매트 비비쿠션 (본품+리필) 0
다슈 맨즈 멀티 커버 스틱 파운데이션 12g 뉴트럴베이지 기획(+미니 톤 컨트롤 로션) 0
두잉왓 소프트 블러링 필터 쿠션 15g 단품/기획 0


In [13]:
for i in df2['상품이름'].unique():
    cnt = (
        df2[df2['상품이름'] == i]
        .duplicated(subset=['리뷰내용_정제'])
        .sum()
    )
    print(i, cnt)

df2['통합리뷰_중복'] = df2.duplicated(
    subset=['작성자', '리뷰내용_정제']
)



오브제 내추럴 커버 파운데이션 기획세트(+클렌징폼 50ml) 9
비레디 블루 파운데이션 35ml 0
[덱스 PICK]오브제 퍼펙트 옴므 쿠션 15g (SPF50+PA+++) 2
비레디 블루 쿠션 4세대 15g(SPF 34 PA++) 2
다슈 맨즈 멀티 커버 스틱 파운데이션 ( 단품/기획 ) 0
[맨즈파데1등] 그라펜 팁 스케치 멀티 커버 파운데이션 20ml 0
[착붙쿠션] 그라펜 핸섬 커버쿠션 2종 택1 1
[덱스 PICK] 오브제 내추럴 커버 파운데이션 13g (SPF50+, PA++++) 4
비레디 블루 쿠션 4세대 리필 15g 1
다슈 맨즈 아쿠아 매트 비비 쿠션 15g 0
[피부착붙컬러] 그라펜 매치업 파운데이션 40ml 단품/기획 0
다슈 맨즈 아쿠아 매트 비비 쿠션 (리필용) 0
다슈 맨즈 아쿠아 매트 비비쿠션 (본품+리필) 0
다슈 맨즈 멀티 커버 스틱 파운데이션 12g 뉴트럴베이지 기획(+미니 톤 컨트롤 로션) 0
두잉왓 소프트 블러링 필터 쿠션 15g 단품/기획 0


In [ ]:
df2.duplicated('리뷰내용').sum() - df2.duplicated(['상품이름', '리뷰내용']).sum()


In [15]:
df2_clean = df2.drop_duplicates(
    subset=['작성자', '리뷰내용_정제'],
    keep='first'
).reset_index(drop=True)
